# Before: 1994~2010년 기준 
* 절도범: 비율 및 횟수 동일하게 증가(절도범 유형만 따로 1994~2010년도 범위에서 분석하기)    

### * 초기설정

In [1]:
import mysql.connector

import pandas as pd
import numpy as np
import folium
import seaborn as sns
import koreanize_matplotlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
lsw = mysql.connector.connect(
host = "database-1.cz44qgu84nin.ap-northeast-2.rds.amazonaws.com",
user = "sw",
password = "1234",
database = "edabase"
)

cursor = lsw.cursor(dictionary=True ,buffered=True)
cursor.execute("use edabase")

In [3]:
cursor.execute("show tables")
cursor.fetchall()

[{'Tables_in_edabase': 'crime_age_after'},
 {'Tables_in_edabase': 'crime_age_before'},
 {'Tables_in_edabase': 'crime_count_after'},
 {'Tables_in_edabase': 'crime_count_before'},
 {'Tables_in_edabase': 'crime_dosi_after'},
 {'Tables_in_edabase': 'crime_dosi_before'},
 {'Tables_in_edabase': 'crime_dotweek_after'},
 {'Tables_in_edabase': 'crime_dotweek_before'},
 {'Tables_in_edabase': 'crime_edu_before'},
 {'Tables_in_edabase': 'crime_motive_after'},
 {'Tables_in_edabase': 'crime_motive_before'},
 {'Tables_in_edabase': 'crime_place_after'},
 {'Tables_in_edabase': 'crime_place_before'},
 {'Tables_in_edabase': 'crime_time_after'},
 {'Tables_in_edabase': 'crime_time_before'},
 {'Tables_in_edabase': 'driver_traffic_order'},
 {'Tables_in_edabase': 'walking_order'}]

### * 절도범 유형만 따로 1994~2010년도 범위에서 분석하기

##### [ 지역 ]

In [4]:
cursor.execute("select * from crime_dosi_before")

dosi = cursor.fetchall()
dosi_df_start = pd.DataFrame(dosi)
dosi_df_start

,대,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,서울특별시,346198,350674,347022,349602,377918,349658,...,407157,371736,378072,396355,353242,347290,356719,392626,405857,348541
1,총범죄,소계,소계,부산광역시,108512,113321,117861,118091,138857,134248,...,133624,121514,136483,155995,122499,126900,141593,170436,176217,130047
2,총범죄,소계,소계,대구광역시,69962,72023,76304,83084,97816,93489,...,102852,105298,101180,110640,90386,89404,93814,106545,97603,96342
3,총범죄,소계,소계,인천광역시,66975,63755,73695,84735,103836,95142,...,102654,111568,108207,104419,100871,108648,102302,108802,106071,86782
4,총범죄,소계,소계,광주광역시,38927,43409,49746,52208,64698,56147,...,65979,65028,67753,58808,57898,54954,58955,60902,60785,62091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10051,특별법범,기타특별법,소계,제천시,0,0,0,0,0,0,...,0,0,0,19,16,15,0,0,0,0
10052,특별법범,기타특별법,소계,논산시,0,0,0,0,0,0,...,0,0,0,38,54,167,0,0,0,0
10053,특별법범,기타특별법,소계,기타도시,773,1722,1843,5706,5060,5944,...,11199,15231,18009,1726,2209,2346,0,0,0,0
10054,특별법범,기타특별법,소계,도시이외,4253,4514,4815,7039,9940,12604,...,30857,43130,47955,1473,1255,2193,0,0,0,0


In [5]:
dosi_df_start.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10056 entries, 0 to 10055
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대       10056 non-null  object
 1   중       10056 non-null  object
 2   소       10056 non-null  object
 3   발생장소    10056 non-null  object
 4   1994년   10056 non-null  int64 
 5   1995년   10056 non-null  int64 
 6   1996년   10056 non-null  int64 
 7   1997년   10056 non-null  int64 
 8   1998년   10056 non-null  int64 
 9   1999년   10056 non-null  int64 
 10  2000년   10056 non-null  int64 
 11  2001년   10056 non-null  int64 
 12  2002년   10056 non-null  int64 
 13  2003년   10056 non-null  int64 
 14  2004년   10056 non-null  int64 
 15  2005년   10056 non-null  int64 
 16  2006년   10056 non-null  int64 
 17  2007년   10056 non-null  int64 
 18  2008년   10056 non-null  int64 
 19  2009년   10056 non-null  int64 
 20  2010년   10056 non-null  int64 
dtypes: int64(17), object(4)
memory usage: 1.6+ MB


In [6]:
dosi_df = dosi_df_start[dosi_df_start["중"] == "절도범"]
dosi_df.reset_index(inplace=True, drop=True)

In [7]:
dosi_df.drop(columns="대", inplace=True)
dosi_df

/tmp/ipykernel_37862/3166770567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dosi_df.drop(columns="대", inplace=True)


,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,서울특별시,15389,15701,14053,12883,15658,14488,35927,52309,61444,57332,36266,34057,29264,29973,28900,37212,49396
1,절도범,소계,부산광역시,5564,7649,7753,8589,11352,11013,13648,12268,13386,14803,11939,11244,12741,12854,14469,22077,22300
2,절도범,소계,대구광역시,3052,2886,4039,4843,5466,6256,9450,7298,5844,6973,5615,8903,9977,8644,8857,10319,15643
3,절도범,소계,인천광역시,3216,2583,4874,6687,5549,3930,10997,8325,9548,10007,8676,13595,13025,11227,10822,11358,10587
4,절도범,소계,광주광역시,3375,4799,5355,5807,4980,2898,7687,8112,7585,7919,6044,9173,9489,10715,12179,12288,12631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,절도범,상습절도,천안시,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
216,절도범,상습절도,춘천시,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
217,절도범,상습절도,원주시,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
218,절도범,상습절도,기타도시,104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
dosi_df_total = dosi_df[dosi_df["소"] == "소계"]
dosi_df_total

,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,서울특별시,15389,15701,14053,12883,15658,14488,35927,52309,61444,57332,36266,34057,29264,29973,28900,37212,49396
1,절도범,소계,부산광역시,5564,7649,7753,8589,11352,11013,13648,12268,13386,14803,11939,11244,12741,12854,14469,22077,22300
2,절도범,소계,대구광역시,3052,2886,4039,4843,5466,6256,9450,7298,5844,6973,5615,8903,9977,8644,8857,10319,15643
3,절도범,소계,인천광역시,3216,2583,4874,6687,5549,3930,10997,8325,9548,10007,8676,13595,13025,11227,10822,11358,10587
4,절도범,소계,광주광역시,3375,4799,5355,5807,4980,2898,7687,8112,7585,7919,6044,9173,9489,10715,12179,12288,12631
5,절도범,소계,대전광역시,2454,2656,3182,4552,3923,5502,7489,6397,5344,5831,4431,4937,6806,8278,9326,10697,8746
6,절도범,소계,울산광역시,1083,1289,1888,3603,2981,4040,4641,4627,4767,4259,3446,4780,4738,8831,7521,7557,7644
7,절도범,소계,부천시,1201,818,988,737,947,1196,3070,3093,2405,3018,1851,6819,7022,6678,7884,7548,6064
8,절도범,소계,수원시,1916,1781,1830,2299,2264,1746,2763,2997,3135,4440,5359,5833,5348,4465,5863,7014,5909
9,절도범,소계,성남시,999,921,1076,1398,1692,1843,3070,2847,2150,3040,1925,3761,4500,4698,4054,6388,4743


In [16]:
df_melted = dosi_df_total.melt(id_vars=["중", "소", "발생장소"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="발생장소", title="[ 년도별 & 지역별 발생한 절도범 건수 ]", markers=True)
fig.show()

##### [ 장소 ]

In [11]:
cursor.execute("select * from crime_place_before")

place = cursor.fetchall()
place_df_start = pd.DataFrame(place)
place_df_start

,대,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,아파트연립다세대,29040,29768,34854,38995,51304,64405,...,78803,67524,75696,51569,56882,57995,60223,67220,69407,68193
1,총범죄,소계,소계,단독주택,69758,63390,63857,60788,67878,72176,...,77922,63811,63163,101715,99660,98633,103025,114438,121803,104185
2,총범죄,소계,소계,고속도로,18853,15964,17913,17333,14100,19700,...,22444,29653,36181,36829,27189,26444,27722,26722,26007,24094
3,총범죄,소계,소계,노 상,613625,588136,657907,770681,829448,777313,...,952832,955185,945084,1025940,883456,827277,943599,1059330,926208,840053
4,총범죄,소계,소계,상 점,51137,60577,69620,62469,73883,73310,...,75281,75391,80796,71676,56938,60784,63121,58334,71627,67420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,특별법범,기타특별법,소계,해 상,2466,3389,3601,15750,24136,20049,...,14801,16898,15380,1575,2514,3339,2877,1609,1280,2704
4812,특별법범,기타특별법,소계,부 대,6,30,13,10,10,18,...,20,37,33,22,9,9,14,21,21,13
4813,특별법범,기타특별법,소계,구금장소,4,9,2,11,12,9,...,93,36,7,1,0,2,0,0,0,1
4814,특별법범,기타특별법,소계,공 지,132,146,163,208,217,184,...,471,490,537,253,219,194,161,138,119,129


In [17]:
place_df = place_df_start[place_df_start["중"] == "절도범"]
place_df.reset_index(inplace=True, drop=True)
place_df

,대,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,형법범,절도범,소계,아파트연립다세대,5571,5512,6951,8616,9658,10337,...,27728,23031,24854,12981,19935,18744,20601,21125,22283,25624
1,형법범,절도범,소계,단독주택,9429,8292,9176,10263,11012,11894,...,21957,16588,16288,22743,27590,27251,31328,32560,36314,38158
2,형법범,절도범,소계,고속도로,52,38,40,54,46,57,...,114,118,151,113,155,343,143,97,103,150
3,형법범,절도범,소계,노 상,18892,22092,25248,30323,30734,25221,...,53168,49064,50987,42471,50746,53182,56690,58195,62288,62806
4,형법범,절도범,소계,상 점,6227,5657,6280,6916,8201,8928,...,16778,21099,21115,16388,17280,18868,21209,22704,31062,30156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,형법범,절도범,상습절도,산 야,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162,형법범,절도범,상습절도,해 상,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163,형법범,절도범,상습절도,부 대,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,형법범,절도범,상습절도,공 지,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
place_df_total = place_df[place_df["소"] == "소계"]
place_df_total.drop(columns="대", inplace=True)
place_df_total.reset_index(inplace=True, drop=True)
place_df_total

/tmp/ipykernel_37862/2361322919.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,중,소,발생장소,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,아파트연립다세대,5571,5512,6951,8616,9658,10337,25193,27728,23031,24854,12981,19935,18744,20601,21125,22283,25624
1,절도범,소계,단독주택,9429,8292,9176,10263,11012,11894,23273,21957,16588,16288,22743,27590,27251,31328,32560,36314,38158
2,절도범,소계,고속도로,52,38,40,54,46,57,133,114,118,151,113,155,343,143,97,103,150
3,절도범,소계,노 상,18892,22092,25248,30323,30734,25221,52315,53168,49064,50987,42471,50746,53182,56690,58195,62288,62806
4,절도범,소계,상 점,6227,5657,6280,6916,8201,8928,15911,16778,21099,21115,16388,17280,18868,21209,22704,31062,30156
5,절도범,소계,시장·노점,686,529,575,598,650,977,1106,1049,1376,1286,814,712,596,628,607,766,1252
6,절도범,소계,숙박업소목욕탕,1994,1958,2185,2418,2193,2162,4707,5849,5523,6265,5978,7402,7179,7644,7588,8561,9267
7,절도범,소계,유흥접객업소,1845,1603,1431,1463,1358,1713,2861,3503,3307,2821,6442,8374,8687,9417,10544,15218,16146
8,절도범,소계,사무실,3110,3425,3703,4388,4569,4428,8796,9198,8449,9418,6747,7568,6784,7150,8044,7867,8493
9,절도범,소계,공 장,1141,1011,1051,1138,1583,1332,2151,1990,1708,2119,1807,2178,2178,3333,3696,2827,3561


In [19]:
df_melted = place_df_total.melt(id_vars=["중", "소", "발생장소"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="발생장소", title="[ 장소별 발생한 절도범 건수 ]", markers=True)
fig.show()

##### [ 요일 ]

In [20]:
cursor.execute("select * from crime_dotweek_before")

dotweek = cursor.fetchall()
dotweek_df_start = pd.DataFrame(dotweek)
dotweek_df_start

,대,중,소,요일별,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,금요일,155783,186346,186702,195362,223257,199731,...,220963,213900,205870,292731,263790,260812,278751,311326,308390,271056
1,총범죄,소계,소계,목요일,150113,172499,173836,194756,209550,195040,...,208664,198683,188425,289785,256386,258618,275767,309426,304192,269117
2,총범죄,소계,소계,미상,236241,40965,23124,43892,43303,51797,...,146888,187334,277639,0,0,0,0,0,0,0
3,총범죄,소계,소계,수요일,184010,239726,311642,339724,385102,389466,...,412886,406079,389249,289198,251979,256370,272834,301095,303618,260217
4,총범죄,소계,소계,월요일,163222,204418,233873,252088,285255,260266,...,271643,266589,274494,283695,250836,245820,260188,294789,294054,260612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,형법범,풍속범,소계,수요일,2245,2969,3179,3348,3534,4324,...,4147,3815,2932,2070,1541,2897,1944,1943,5088,2468
2046,형법범,풍속범,소계,월요일,1991,2442,2756,2556,2646,2793,...,2688,2534,2074,1934,1621,2826,1884,2007,5427,2280
2047,형법범,풍속범,소계,일요일,1449,1578,1418,1213,1390,1502,...,1640,1380,1056,1769,1472,2016,1791,1820,4179,1858
2048,형법범,풍속범,소계,토요일,1748,1977,1676,1515,1759,2044,...,1932,1591,1201,2272,1779,2572,2010,2226,4306,2138


In [21]:
dotweek_df = dotweek_df_start[dotweek_df_start["중"] == "절도범"]
dotweek_df.reset_index(inplace=True, drop=True)
dotweek_df

,대,중,소,요일별,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,형법범,절도범,상습절도,금요일,98,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,형법범,절도범,상습절도,목요일,55,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,형법범,절도범,상습절도,미상,170,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,형법범,절도범,상습절도,수요일,74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,형법범,절도범,상습절도,월요일,72,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,형법범,절도범,상습절도,일요일,58,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,형법범,절도범,상습절도,토요일,69,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,형법범,절도범,상습절도,화요일,45,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,형법범,절도범,소계,금요일,6870,8321,8826,10346,11373,10086,...,21255,19364,19535,22549,28476,28943,32120,33312,38636,40101
9,형법범,절도범,소계,목요일,6695,7969,8598,10373,11383,10247,...,19766,18097,18431,22727,26950,27943,30438,32345,36574,38837


In [22]:
dotweek_df_total = dotweek_df[dotweek_df["소"] == "소계"]
dotweek_df_total.drop(columns="대", inplace=True)
dotweek_df_total.reset_index(inplace=True, drop=True)
dotweek_df_total = dotweek_df_total.reindex([4,7,3,1,0,6,5,2])
dotweek_df_total.reset_index(inplace=True, drop=True)
dotweek_df_total

/tmp/ipykernel_37862/2733080067.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,중,소,요일별,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,월요일,7461,9574,11252,13277,15214,14241,27547,26084,26789,28012,22385,27141,26643,29538,31699,36801,38334
1,절도범,소계,화요일,6991,9081,9476,11551,12009,11478,23148,23716,21632,21967,22266,27226,27374,30003,32877,35417,38990
2,절도범,소계,수요일,7745,10170,14088,16205,18385,20049,37782,39575,40137,39929,22357,27137,28126,30518,31585,38113,37504
3,절도범,소계,목요일,6695,7969,8598,10373,11383,10247,20973,19766,18097,18431,22727,26950,27943,30438,32345,36574,38837
4,절도범,소계,금요일,6870,8321,8826,10346,11373,10086,20950,21255,19364,19535,22549,28476,28943,32120,33312,38636,40101
5,절도범,소계,토요일,7305,8381,8955,10252,10701,10877,19799,19224,16012,15942,23232,28026,29266,31585,32833,37536,40575
6,절도범,소계,일요일,6387,7323,7821,8772,9855,8834,16721,16807,14214,14963,19795,23824,24375,28256,28565,33346,35069
7,절도범,소계,미상,10801,1891,1222,2287,2518,3583,6956,14277,22212,28573,0,0,0,0,0,0,0


In [23]:
df_melted = dotweek_df_total.melt(id_vars=["중", "소", "요일별"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="요일별", title="[ 요일별 발생한 절도범 건수 ]", markers=True)
fig.show()

##### [ 시간 ]

In [24]:
cursor.execute("select * from crime_time_before")

time = cursor.fetchall()
time_df_start = pd.DataFrame(time)
time_df_start

,대,중,소,발생시간,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,심야(00:00~04:00),116664,125759,135562,145650,181537,199965,...,231374,227356,232135,794266,669626,658792,682121,825648,838369,657125
1,총범죄,소계,소계,새벽(04:00~07:00),38998,41308,43568,47170,48238,54892,...,64021,61087,60858,92146,83080,79152,90689,99263,94166,87811
2,총범죄,소계,소계,오전(07:00~12:00),163507,180626,192047,195345,230522,231894,...,258970,235199,234504,206451,200434,207161,223647,243143,242713,231321
3,총범죄,소계,소계,오후(12:00~18:00),248225,247811,258781,259432,294722,300942,...,299365,287782,285512,301583,272648,278197,298859,332056,316751,298739
4,총범죄,소계,소계,초저녁(18:00~20:00),91141,94094,94087,102344,118736,114916,...,121742,115557,117096,104313,98884,98744,104802,113639,113412,113618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,특별법범,기타특별법,소계,오전(07:00~12:00),2530,3625,3886,7636,10823,13079,...,19494,27573,26237,1932,2968,3061,3756,4090,3981,4294
1830,특별법범,기타특별법,소계,오후(12:00~18:00),2535,3164,3506,6126,9500,9320,...,25198,34700,35638,1758,1767,2157,2664,5274,4344,3733
1831,특별법범,기타특별법,소계,초저녁(18:00~20:00),401,575,589,1832,1982,1927,...,10325,12166,17555,378,690,530,378,795,608,486
1832,특별법범,기타특별법,소계,밤(20:00~24:00),322,557,777,1793,2044,5161,...,43952,61550,84536,1216,742,877,1057,891,743,824


In [25]:
time_df = time_df_start[time_df_start["중"] == "절도범"]
time_df.reset_index(inplace=True, drop=True)
time_df

,대,중,소,발생시간,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,형법범,절도범,소계,심야(00:00~04:00),10772,11609,12214,13829,16817,16923,...,26589,26527,27148,38555,43296,43641,49638,58482,76692,71944
1,형법범,절도범,소계,새벽(04:00~07:00),2512,2816,3246,3876,3831,3945,...,8208,7039,7754,9830,11979,11936,12778,13249,14641,15402
2,형법범,절도범,소계,오전(07:00~12:00),8414,9434,10896,13179,14693,14216,...,32837,30728,32747,24138,32036,32649,36209,36292,39723,43782
3,형법범,절도범,소계,오후(12:00~18:00),13292,13968,15350,18006,19543,18746,...,37185,37219,38784,36568,43566,44265,49471,50038,56345,61137
4,형법범,절도범,소계,초저녁(18:00~20:00),4581,5689,6043,7099,7370,6751,...,15128,14067,14361,15225,19429,20105,22139,23148,23916,27298
5,형법범,절도범,소계,밤(20:00~24:00),9314,10529,11783,13080,13009,12271,...,24052,20796,23065,30995,38474,40074,42223,42007,45106,49847
6,형법범,절도범,소계,미상,11370,8665,10706,13994,16175,16543,...,36705,42081,43493,0,0,0,0,0,0,0
7,형법범,절도범,절도,심야(00:00~04:00),4917,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,형법범,절도범,절도,새벽(04:00~07:00),1785,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,형법범,절도범,절도,오전(07:00~12:00),6256,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
time_df_total = time_df[time_df["소"] == "소계"]
time_df_total.drop(columns="대", inplace=True)
time_df_total.reset_index(inplace=True, drop=True)
time_df_total

/tmp/ipykernel_37862/2570507428.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,중,소,발생시간,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,심야(00:00~04:00),10772,11609,12214,13829,16817,16923,27183,26589,26527,27148,38555,43296,43641,49638,58482,76692,71944
1,절도범,소계,새벽(04:00~07:00),2512,2816,3246,3876,3831,3945,8230,8208,7039,7754,9830,11979,11936,12778,13249,14641,15402
2,절도범,소계,오전(07:00~12:00),8414,9434,10896,13179,14693,14216,30704,32837,30728,32747,24138,32036,32649,36209,36292,39723,43782
3,절도범,소계,오후(12:00~18:00),13292,13968,15350,18006,19543,18746,36958,37185,37219,38784,36568,43566,44265,49471,50038,56345,61137
4,절도범,소계,초저녁(18:00~20:00),4581,5689,6043,7099,7370,6751,14913,15128,14067,14361,15225,19429,20105,22139,23148,23916,27298
5,절도범,소계,밤(20:00~24:00),9314,10529,11783,13080,13009,12271,24551,24052,20796,23065,30995,38474,40074,42223,42007,45106,49847
6,절도범,소계,미상,11370,8665,10706,13994,16175,16543,31337,36705,42081,43493,0,0,0,0,0,0,0


In [27]:
df_melted = time_df_total.melt(id_vars=["중", "소", "발생시간"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="발생시간", title="[ 발생시간별 발생한 절도범 건수 ]", markers=True)
fig.show()

##### [ 나이 ]

In [28]:
cursor.execute("select * from crime_age_before")

age = cursor.fetchall()
age_df_start = pd.DataFrame(age)
age_df_start

,대,중,소,연령별,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,미성년,108681,123372,136323,153380,151383,141519,...,131059,115210,96697,86861,83477,90628,115661,123044,118058,94862
1,총범죄,소계,소계,14세미만,2648,2109,2149,1875,1440,1339,...,1378,1667,1376,1503,1679,1718,2602,5547,2615,621
2,총범죄,소계,소계,성인,1302941,1325225,1412545,1516202,1707707,1777161,...,1841330,1821169,1811469,2093377,1838155,1858300,1955214,2144039,2151798,1836620
3,총범죄,소계,소계,미상,11996,1562,2532,2773,3906,7478,...,7785,6608,9044,87666,48194,43849,38670,66037,63859,54837
4,형법범,소계,소계,미성년,78972,84998,86611,99519,103243,94310,...,84518,75511,67408,61577,62008,66740,80008,75331,86087,74368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,특별법범,후천성면역결핍증예방법,소계,성인,0,0,0,1,0,0,...,0,4,1,1,1,4,0,0,0,0
845,특별법범,기타특별법,소계,미성년,123,156,247,349,343,501,...,9346,11129,9059,64,54,46,115,172,174,153
846,특별법범,기타특별법,소계,14세미만,16,8,11,9,5,12,...,68,119,113,7,0,1,0,3,3,0
847,특별법범,기타특별법,소계,성인,15720,20643,20759,25204,28168,46281,...,169612,234276,282003,17279,15560,17812,19456,30305,41471,39456


In [29]:
age_df = age_df_start[age_df_start["중"] == "절도범"]
age_df.reset_index(inplace=True, drop=True)
age_df

,대,중,소,연령별,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,형법범,절도범,소계,미성년,29225,30181,31392,34201,41063,32898,...,25188,26345,24208,23417,27367,30652,38143,36044,43549,37069
1,형법범,절도범,소계,14세미만,1130,906,878,731,598,453,...,425,702,412,602,723,923,1305,986,512,201
2,형법범,절도범,소계,성인,23863,21808,20573,21973,31970,33613,...,34035,36962,37104,40590,42706,46600,53452,58546,79269,79148
3,형법범,절도범,소계,미상,283,31,65,116,161,300,...,249,337,339,4391,2076,1763,1613,1728,3266,2893
4,형법범,절도범,절도,미성년,6962,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,형법범,절도범,절도,14세미만,150,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,형법범,절도범,절도,성인,14427,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,형법범,절도범,절도,미상,106,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,형법범,절도범,특가법(절도),미성년,1504,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,형법범,절도범,특가법(절도),14세미만,63,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
age_df_total = age_df[age_df["소"] == "소계"]
age_df_total.drop(columns="대", inplace=True)
age_df_total.reset_index(inplace=True, drop=True)
age_df_total

/tmp/ipykernel_37862/3637754249.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,중,소,연령별,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,미성년,29225,30181,31392,34201,41063,32898,32668,25188,26345,24208,23417,27367,30652,38143,36044,43549,37069
1,절도범,소계,14세미만,1130,906,878,731,598,453,477,425,702,412,602,723,923,1305,986,512,201
2,절도범,소계,성인,23863,21808,20573,21973,31970,33613,33112,34035,36962,37104,40590,42706,46600,53452,58546,79269,79148
3,절도범,소계,미상,283,31,65,116,161,300,302,249,337,339,4391,2076,1763,1613,1728,3266,2893


In [31]:
df_melted = age_df_total.melt(id_vars=["중", "소", "연령별"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="연령별", title="[ 연령별 발생한 절도범 건수 ]", markers=True)
fig.show()

##### [ 범죄동기 ]

In [32]:
cursor.execute("select * from crime_motive_before")

motive = cursor.fetchall()
motive_df_start = pd.DataFrame(motive)
motive_df_start

,대,중,소,범행동기,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,총범죄,소계,소계,이욕,184442,187138,191339,185785,226853,241986,...,220235,213188,217442,83540,73513,79960,83286,102075,124558,290974
1,총범죄,소계,소계,사행심,18824,20298,21047,21898,21103,21500,...,13858,20796,16405,26061,22196,31133,26597,31240,42330,562
2,총범죄,소계,소계,보복,6123,4789,4711,5733,5129,5323,...,4881,4264,3347,5543,4781,4421,4438,2081,1432,26164
3,총범죄,소계,소계,가정불화,5820,4479,4340,3679,6131,11404,...,12076,9976,10686,10636,8693,7890,7286,5469,4159,5348
4,총범죄,소계,소계,호기심,15272,17840,26890,32307,24490,14962,...,17429,17110,13504,15828,17762,25468,30557,33906,49894,25851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174,특별법범,기타특별법,소계,유혹,64,57,36,64,52,356,...,1594,1337,1167,69,36,51,101,301,382,211
2175,특별법범,기타특별법,소계,우발적,169,283,346,506,371,760,...,13263,23018,22295,526,243,328,334,581,1093,766
2176,특별법범,기타특별법,소계,현실불만,24,35,46,108,132,129,...,282,382,680,221,105,42,67,69,99,97
2177,특별법범,기타특별법,소계,부주의,1491,2554,2296,4108,3243,5753,...,45962,58749,84826,1632,1227,1381,1539,2960,3911,3857


In [33]:
motive_df = motive_df_start[motive_df_start["중"] == "절도범"]
motive_df.reset_index(inplace=True, drop=True)
motive_df

,대,중,소,범행동기,1994년,1995년,1996년,1997년,1998년,1999년,...,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,형법범,절도범,소계,이욕,22267,20148,19751,20335,28156,27224,...,24623,25769,23964,12746,13015,13012,15196,17350,21761,36131
1,형법범,절도범,소계,사행심,1002,859,854,944,974,887,...,822,623,512,1225,1510,1659,1618,2078,3424,10
2,형법범,절도범,소계,보복,107,88,71,86,112,116,...,98,91,95,97,107,111,118,7,24,2023
3,형법범,절도범,소계,가정불화,101,90,57,45,65,82,...,67,74,61,78,90,102,95,56,57,75
4,형법범,절도범,소계,호기심,6020,7301,7701,8924,9011,6187,...,5003,6226,6061,7232,8563,9763,10952,9430,9420,8058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,형법범,절도범,상습절도,호기심,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,형법범,절도범,상습절도,유혹,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,형법범,절도범,상습절도,우발적,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,형법범,절도범,상습절도,부주의,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
motive_df_total = motive_df[motive_df["소"] == "소계"]
motive_df_total.drop(columns="대", inplace=True)
motive_df_total.reset_index(inplace=True, drop=True)
motive_df_total

/tmp/ipykernel_37862/4217943652.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,중,소,범행동기,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도범,소계,이욕,22267,20148,19751,20335,28156,27224,26733,24623,25769,23964,12746,13015,13012,15196,17350,21761,36131
1,절도범,소계,사행심,1002,859,854,944,974,887,915,822,623,512,1225,1510,1659,1618,2078,3424,10
2,절도범,소계,보복,107,88,71,86,112,116,95,98,91,95,97,107,111,118,7,24,2023
3,절도범,소계,가정불화,101,90,57,45,65,82,69,67,74,61,78,90,102,95,56,57,75
4,절도범,소계,호기심,6020,7301,7701,8924,9011,6187,6641,5003,6226,6061,7232,8563,9763,10952,9430,9420,8058
5,절도범,소계,유혹,1033,856,873,1290,1648,1416,1479,1160,1474,1229,1557,1623,2128,2753,3528,4776,3207
6,절도범,소계,우발적,7674,7777,7552,7479,9953,9492,9894,9555,9521,9843,13051,14782,18000,22798,27783,41186,34070
7,절도범,소계,현실불만,225,173,239,263,226,192,195,163,167,203,214,254,329,403,541,809,567
8,절도범,소계,부주의,463,624,707,851,1057,1167,1057,871,895,1102,579,706,803,1015,1550,2103,2144
9,절도범,소계,기타,14479,14104,14225,16073,21992,20048,19004,17110,18804,18581,31619,31499,33108,38260,33995,42524,32825


In [35]:
df_melted = motive_df_total.melt(id_vars=["중", "소", "범행동기"], var_name="년도", value_name="발생건수")

fig = px.line(df_melted, x="년도", y="발생건수", color="범행동기", title="[ 범행동기별 발생한 절도범 건수 ]", markers=True)
fig.show()

In [36]:
lsw.close()